# Plot Hubble diagram

RAISINs and/or low-z samples

    Version 1.5
- including the distance moduli of RAISIN-1 optical data estimated from SNANA.

# User interface

In [1]:
# Peculiar velocity. This is used only in the part where I compute
# the intrinsic dispersion. 150 km/s is the value I used to GP fit each SN
sigma_vpec = 150     # km/s. Usual options (150, 200, 300, 400)

#--------------------------------------------

# Samples to plot: RAISIN-1 and or RAISIN-2
LowzSample=False
RAISIN_1_Sample=False
RAISIN_2_Sample=False
RAISIN1_Snana = True

# LOW-Z sub-sample:
# Sample = 'CfA'
# Sample = 'CSP'
# Sample = 'Others'
Sample = 'AllSamples'

BandType = 'Optical'
# BandType = 'OpticalNIR'

#----------------------

PrintNameOutliers = True
PrintRMS = False
"""
PrintNameOutliers = False
PrintRMS = False
"""

#------ (Fixed) ----------------

# Intervals for plotting the Hubble diagram
# x_interval = (0, 0.065)
# x_interval = (0, 0.7)

# y_interval = (-1.2, 1.2)
y_interval = (-1, 1)

#----------------------

# Quality Cutoffs for the low-z sample
zcmbMin = 0.0;  zcmbMax = 0.06
dm15Min = 0.80;  dm15Max = 1.60
EBVhostMax = 0.4
EBV_mwMax  = 1

################################

#--- Fixed values ---
cc = 299792.458  # Speed of light (km/s)


In [2]:
# Defining the directories
MainDir = '/Users/arturo/Dropbox/Research/Articulos/12_RAISINs/RAISINs/'

Dir_raisin_1 = MainDir+'RAISIN_1/Data/2017/Snoopy/'+BandType+'/Fit/'
Dir_raisin_2 = MainDir+'RAISIN_2/Data/2017_03_30/Snoopy_zp27.5/'+BandType+'/Fit/'
Dir_raisin1_snana = MainDir+'RAISIN_1/Data/2017/DavidJones/SNANA_SALT2_fit/'

DirSaveOutput= MainDir+'Plots/HubbleDiag/'

# Low-z data
DirLowzData = '/Users/arturo/Dropbox/Research/SoftwareResearch/Snoopy/AndyLCComp/\
all/Snoopy_FromMag/2_4RAISINs_old/' 
DirMuData_lowz = DirLowzData+Sample+'/'+BandType+'/'

# Visualizing the directories
print '# Directories of the distance-modulus files for the low-z sample:'
print DirMuData_lowz
print ' '
print '# RAISIN-1:'
print Dir_raisin_1
print ' '
print '# RAISIN-2:'
print Dir_raisin_2
print ' '
print '# RAISIN-1 (SNANA, optical):'
print Dir_raisin1_snana

# Directories of the distance-modulus files for the low-z sample:
/Users/arturo/Dropbox/Research/SoftwareResearch/Snoopy/AndyLCComp/all/Snoopy_FromMag/2_4RAISINs_old/AllSamples/Optical/
 
# RAISIN-1:
/Users/arturo/Dropbox/Research/Articulos/12_RAISINs/RAISINs/RAISIN_1/Data/2017/Snoopy/Optical/Fit/
 
# RAISIN-2:
/Users/arturo/Dropbox/Research/Articulos/12_RAISINs/RAISINs/RAISIN_2/Data/2017_03_30/Snoopy_zp27.5/Optical/Fit/
 
# RAISIN-1 (SNANA, optical):
/Users/arturo/Dropbox/Research/Articulos/12_RAISINs/RAISINs/RAISIN_1/Data/2017/DavidJones/SNANA_SALT2_fit/


------

# Automatic

In [3]:
import numpy as np
from matplotlib import pyplot as plt
from scipy.integrate import quad as intquad

# import os
# import glob # To read the name of the files in a given directory

In [4]:
# Defining some values
if RAISIN1_Snana == True:
    HoFix = 70  
    OmMFix = 0.3 # Omega_Matter
    OmLFix = 0.7 # Omega_Lambda
    wFix = -1 # Dark energy EoS
else:
    HoFix = 72  
    OmMFix = 0.28 # Omega_Matter
    OmLFix = 0.72 # Omega_Lambda
    wFix = -1 # Dark energy EoS
    
# Hubble constant (km/(s Mpc)):
#    - 70 : SNANA-SALT2 default value
#    - 72 : SNoopy  default value
#    - 73.24: Adam Riess Cepheid values. This is what I use in the low-z paper.

### Defining directories

In [5]:
# Create directory

#- Force the creation of the directory to save the outputs.
#- "If the subdirectory does not exist then create it"
import os # To use command line like instructions
if not os.path.exists(DirSaveOutput): os.makedirs(DirSaveOutput)


# Visualizing the directories
print 'Dir to save the outputs:'
print DirSaveOutput
print ' '

Dir to save the outputs:
/Users/arturo/Dropbox/Research/Articulos/12_RAISINs/RAISINs/Plots/HubbleDiag/
 


In [6]:
# Read the distance modulus file

""" 
Mu_lowz = np.genfromtxt(DirMuData_lowz+'DistanceModulus_.txt', 
                dtype=['S32', float,float,float,float,float,float,float,float,
                              float,float,float,float,float,float,float,float])

Mu_raisin1 = np.genfromtxt(Dir_raisin_1+'DistanceModulus_.txt', 
                dtype=['S32', float,float,float,float,float,float,float,float,
                              float,float,float,float,float,float,float,float])

Mu_raisin2 = np.genfromtxt(Dir_raisin_2+'DistanceModulus_.txt', 
                dtype=['S32', float,float,float,float,float,float,float,float,
                              float,float,float,float,float,float,float,float])

"""

Mu_raisin1Snana = np.genfromtxt(Dir_raisin1_snana+'SALT2mu.fitres', skip_header=2,
                dtype=['S4', 'S13', 
                      float, float, float, float, float, float, float, float, float, float,
                      float, float, float, float, float, float, float, float, float, float,
                      float, float, float, float, float, float, float, float, float])

In [7]:
Mu_raisin1Snana[0]

('SN:', 'PSc440005', 0.306, 0.001, 2.9024e-05, 1.32e-06, 0.1315, 0.0378, 0.177, 0.4222, 56219.309, 0.42, 21.9781, 0.0484, -2.66e-07, -4.08e-08, 0.00574, 24.4, 40.0, 0.975, 17.94, 13.01, 12.56, 15.0, 0.0, 41.0339, 0.1174, -0.159, -1.357, 0.391, 0.087)

In [18]:
Mu_raisin1Snana['f2']

array([ 0.306 ,  0.433 ,  0.25  ,  0.41  ,  0.331 ,  0.346 ,  0.43  ,
        0.22  ,  0.334 ,  0.422 ,  0.34  ,  0.31  ,  0.325 ,  0.502 ,
        0.308 ,  0.519 ,  0.283 ,  0.41  ,  0.275 ,  0.4776,  0.422 ,
        0.44  ,  0.3   ])

In [9]:
Mu_raisin1Snana['f25']

array([ 41.0339,  41.8202,  40.6124,  41.95  ,  41.2972,  41.3894,
        42.8729,  40.297 ,  41.5641,  41.8326,  40.9941,  41.2138,
        41.3479,  42.2261,  41.4003,  43.9014,  41.0734,  41.8817,
        40.9983,  42.4342,  42.2605,  41.9479,  41.4119])

In [10]:
Mu_raisin1Snana['f26']

array([ 0.1174,  0.1605,  0.0881,  0.2376,  0.1012,  0.2077,  0.5352,
        0.0812,  0.1012,  0.1595,  0.1643,  0.1064,  0.1224,  0.3506,
        0.0912,  0.4471,  0.084 ,  0.1728,  0.0907,  0.1761,  0.2639,
        0.2642,  0.1917])

## The theory

In [11]:
# Inverse of the dimensionless Hubble parameter
def InvEHubblePar(z, OmM, wde):
    "Dimensionless Hubble parameter"
    InvEHubbleParInt = 1.0/(np.sqrt(OmM*(1.0+z)**3.0 + (1.0-OmM)*(1.+z)**(3.*(1.+wde))))
    return InvEHubbleParInt

# ---- The luminosity distance ----
def LumDistanceVec(z, OmM, wde, Ho):
    "Luminosity distance"
    LumDistanceVecInt = 0.
    LumDistanceVecInt = cc*(1.+z)*intquad(InvEHubblePar, 0., z, args=(OmM, wde))[0]/Ho 
    return LumDistanceVecInt

# ---- Distance modulus scalar ----
def DistanceMu(z, OmM, wde, Ho):
    "Distance modulus"     
    DistanceMuInt = 5.0*np.log10(LumDistanceVec(z, OmM, wde, Ho)) + 25.0
    return DistanceMuInt

# ---- Distance modulus Vector ----
def DistanceMuVector(z, OmM, wde, Ho):
    "Distance modulus"     
    DistanceMuInt= []
    for i in range(len(z)):
        DistanceMuInt += [5.0*np.log10(LumDistanceVec(z[i], OmM, wde, Ho)) + 25.0] 
    return DistanceMuInt

#--------------------------------------------------

ztest1 = 0.01

print 'Checking that the functions work well:', DistanceMu(ztest1, OmMFix, wFix, HoFix)
# Checking that the functions work well: 33.1144718537

Checking that the functions work well: 33.1753183809


In [12]:
# sigma^2_mu from the peculiar velocity uncertainty

# This function is used to determine in the sections "Intrinsic dispersion" and "Optical RMS", to
# determine the intrinsic dispersion.

def sigma2_mu_vpec(zcmb, err_zcmb, sigma_vpec):
    sigma2_mu_vpecInt = ((5/(zcmb*np.log(10)))*np.sqrt((sigma_vpec/cc)**2 + err_zcmb**2))**2
    return sigma2_mu_vpecInt

# Test
sigma2_mu_vpec(0.0109942726, 0.0010420420, 150)
# 0.052125037354329877

0.052125037354329877

-------

# Residual Hubble diagram

In [18]:
# To plot the theoretical (spectroscopic) distance modulus
nbins1= 101
z1 = np.linspace(0.001, 0.7, nbins1)
mu1 = DistanceMuVector(z1, OmMFix, wFix, HoFix)


In [19]:
# Compute the residual

muResid_total = []

rms_total=0; count_total = 0

#------ Low-z sample ------

count_lowz = 0; rms_lowz = 0
    
if LowzSample:
    zcmb_lowz_list = []
    muResid_lowz = []
    mu_error_lowz = []

    zcmb = 0; err_zcmb=0; mu = 0; err_mu_snoopy=0; 
    dm15=0; EBVhost=0; EBV_mw=0; error_mu_vpec=0; totalMuError=0;  

    for i in range(len(Mu_lowz)):  
        zcmb = Mu_lowz[i][3]
        err_zcmb = Mu_lowz[i][4]
        mu   = Mu_lowz[i][5]
        err_mu_snoopy = Mu_lowz[i][7]
        dm15 = Mu_lowz[i][9]
        EBVhost = Mu_lowz[i][11]
        EBV_mw = Mu_lowz[i][13]

        error_mu_vpec = np.sqrt(sigma2_mu_vpec(zcmb, err_zcmb, sigma_vpec))

        # Total distance-modulus uncertainty
        totalMuError = np.sqrt(err_mu_snoopy**2 + error_mu_vpec**2)

        # Cutoffs:
        if (zcmb > zcmbMin and zcmb < zcmbMax and 
           dm15>dm15Min and dm15 < dm15Max and
           EBVhost>(-EBVhostMax) and EBVhost < EBVhostMax and EBV_mw < EBV_mwMax):
            zcmb_lowz_list += [zcmb]
            muResid_lowz += [mu - DistanceMu(zcmb, OmMFix, wFix, HoFix)]
            muResid_total += [mu - DistanceMu(zcmb, OmMFix, wFix, HoFix)]
            mu_error_lowz += [totalMuError]
            count_lowz = count_lowz + 1
            count_total = count_total + 1

    # Compute the RMS
    rms_lowz = np.sqrt(np.mean(np.square(muResid_lowz)))
      

#------ RAISIN-1 ------
 
count_raisin1 = 0; rms_raisin1 = 0

if RAISIN_1_Sample:
    zcmb_raisin1_list = []
    muResid_raisin1 = []
    mu_error_raisin1 = []
    name_raisin1 = []

    zcmb = 0; err_zcmb=0; mu = 0; err_mu_snoopy=0; 
    dm15=0; EBVhost=0; EBV_mw=0; error_mu_vpec=0; totalMuError=0; 

    for i in range(len(Mu_raisin1)):      
        zcmb = Mu_raisin1[i][3]
        err_zcmb = Mu_raisin1[i][4]
        mu   = Mu_raisin1[i][5]
        err_mu_snoopy = Mu_raisin1[i][7]

        error_mu_vpec = np.sqrt(sigma2_mu_vpec(zcmb, err_zcmb, sigma_vpec))

        # Total distance-modulus uncertainty
        totalMuError = np.sqrt(err_mu_snoopy**2 + error_mu_vpec**2)

        zcmb_raisin1_list += [zcmb]
        muResid_raisin1 += [mu - DistanceMu(zcmb, OmMFix, wFix, HoFix)]
        muResid_total += [mu - DistanceMu(zcmb, OmMFix, wFix, HoFix)]
        mu_error_raisin1 += [totalMuError]
        count_raisin1 = count_raisin1 + 1
        count_total = count_total + 1

    # Compute the RMS
    rms_raisin1 = np.sqrt(np.mean(np.square(muResid_raisin1)))

        
#------ RAISIN-2 ------

count_raisin2 = 0; rms_raisin2 = 0

if RAISIN_2_Sample:

    zcmb_raisin2_list = []
    muResid_raisin2 = []
    mu_error_raisin2 = []

    zcmb = 0; err_zcmb=0; mu = 0; err_mu_snoopy=0; 
    dm15=0; EBVhost=0; EBV_mw=0; error_mu_vpec=0; totalMuError=0; 

    for i in range(len(Mu_raisin2)):      
        zcmb = Mu_raisin2[i][3]
        err_zcmb = Mu_raisin2[i][4]
        mu   = Mu_raisin2[i][5]
        err_mu_snoopy = Mu_raisin2[i][7]

        error_mu_vpec = np.sqrt(sigma2_mu_vpec(zcmb, err_zcmb, sigma_vpec))

        # Total distance-modulus uncertainty
        totalMuError = np.sqrt(err_mu_snoopy**2 + error_mu_vpec**2)

        zcmb_raisin2_list += [zcmb]
        muResid_raisin2 += [mu - DistanceMu(zcmb, OmMFix, wFix, HoFix)]
        muResid_total += [mu - DistanceMu(zcmb, OmMFix, wFix, HoFix)]
        mu_error_raisin2 += [totalMuError]
        count_raisin2 = count_raisin2 + 1
        count_total = count_total + 1

    # Compute the RMS
    rms_raisin2 = np.sqrt(np.mean(np.square(muResid_raisin2)))

#------------RAISIN -1 SNANA -----------------

count_raisin1Snana = 0; rms_raisin1Snana = 0

if RAISIN1_Snana:

    zcmb_raisin1Snana_list = []
    muResid_raisin1Snana = []
    mu_error_raisin1Snana = []

    zcmb = 0; err_zcmb=0; mu = 0; err_mu=0; 
    dm15=0; EBVhost=0; EBV_mw=0; error_mu_vpec=0; totalMuError=0;     
    
    for i in range(len(Mu_raisin1Snana)):      
        zcmb = Mu_raisin1Snana[i][1]
        err_zcmb = Mu_raisin1Snana[i][2]
        mu   = Mu_raisin1Snana[i][3]
        err_mu = Mu_raisin1Snana[i][4]

        error_mu_vpec = np.sqrt(sigma2_mu_vpec(zcmb, err_zcmb, sigma_vpec))

        # Total distance-modulus uncertainty
        totalMuError = np.sqrt(err_mu**2 + error_mu_vpec**2)

        zcmb_raisin1Snana_list += [zcmb]
        muResid_raisin1Snana += [mu - DistanceMu(zcmb, OmMFix, wFix, HoFix)]
        muResid_total += [mu - DistanceMu(zcmb, OmMFix, wFix, HoFix)]
        mu_error_raisin1Snana += [totalMuError]
        count_raisin1Snana = count_raisin1Snana + 1
        count_total = count_total + 1

    # Compute the RMS
    rms_raisin1Snana = np.sqrt(np.mean(np.square(muResid_raisin1Snana)))

#------------------------  

# Compute total RMS
rms_total = np.sqrt(np.mean(np.square(muResid_total)))


TypeError: ufunc 'multiply' did not contain a loop with signature matching types dtype('S32') dtype('S32') dtype('S32')

### Plotting

In [ ]:
# Defining the x-axis interval to plot

if LowzSample and RAISIN_2_Sample: x_interval = (0, 0.7)
elif LowzSample and RAISIN_1_Sample ==False and RAISIN_2_Sample==False: x_interval = (0, 0.065)
elif LowzSample and RAISIN_1_Sample: x_interval = (0, 0.55)
elif LowzSample==False and RAISIN_1_Sample and RAISIN_2_Sample: x_interval = (0.2, 0.7)
elif LowzSample==False and RAISIN_1_Sample ==False and RAISIN_2_Sample: x_interval = (0.35, 0.7) 
elif LowzSample==False and RAISIN_1_Sample  and RAISIN_2_Sample==False: x_interval = (0.2, 0.55)  


In [ ]:
# Initializing labels
label_0=''; label_1=''; label_2='';

labelOutlierLim = 0.35 # 
labelOutlierLim_lowZ = 4 # 
fontSNname = 10

fig = plt.figure()

# Plotting theory: Flat Universe
mu0 = np.zeros(len(z1))
plt.plot(z1, mu0, color='black', lw=2, alpha=0.5)

#---------------------------
#  Plot the low-z sample

if LowzSample:
    plt.errorbar(zcmb_lowz_list, muResid_lowz, yerr=mu_error_lowz,
                fmt='.', color='green',  ms=10, elinewidth=1.5, capthick=1.5)
    # Label the outliers
    for i in range(len(zcmb_lowz_list)):
        if abs(muResid_lowz[i]) > labelOutlierLim_lowZ and PrintNameOutliers:
            plt.text(zcmb_lowz_list[i]+0.002, muResid_lowz[i]-0.03, Mu_lowz[i][0], 
                     fontsize=fontSNname, color='k')

#---------------------------
#  Plot the RAISIN samples

if count_raisin1 >0:
    plt.errorbar(zcmb_raisin1_list, muResid_raisin1, yerr=mu_error_raisin1,
                fmt='.', color='red',  ms=10, elinewidth=1.5, capthick=1.5)
    
    # Label the outliers
    for i in range(len(zcmb_raisin1_list)):
        if abs(muResid_raisin1[i]) > labelOutlierLim and PrintNameOutliers:
            plt.text(zcmb_raisin1_list[i]+0.002, muResid_raisin1[i]-0.03, Mu_raisin1[i][0], 
                     fontsize=fontSNname, color='k')

            
if count_raisin2 >0:
    plt.errorbar(zcmb_raisin2_list, muResid_raisin2, yerr=mu_error_raisin2,
                fmt='.', color='blue',  ms=10, elinewidth=1.5, capthick=1.5)
    
    # Label the outliers
    for i in range(len(zcmb_raisin2_list)):
        if abs(muResid_raisin2[i]) > labelOutlierLim and PrintNameOutliers:
            plt.text(zcmb_raisin2_list[i]+0.002, muResid_raisin2[i]-0.03, Mu_raisin2[i][0], 
                     fontsize=fontSNname, color='k')   
    
#---------------------------
#  Plot the RAISIN-1 fit from SNANA

if count_raisin1Snana >0:
    plt.errorbar(zcmb_raisin1Snana_list, muResid_raisin1Snana, yerr=mu_error_raisin1Snana,
                fmt='.', color='lime',  ms=10, elinewidth=1.5, capthick=1.5)
    
    # Label the outliers
    for i in range(len(zcmb_raisin1Snana_list)):
        if abs(muResid_raisin1Snana[i]) > labelOutlierLim and PrintNameOutliers:
            plt.text(zcmb_raisin1Snana_list[i]+0.002, muResid_raisin1Snana[i]-0.03, Mu_raisin1Snana[i][0], 
                     fontsize=fontSNname, color='k')

#---------------------------
# Print some texts


# For RAISINs 
if LowzSample and RAISIN_2_Sample:
    plt.text(0.05, y_interval[1]-0.2, '%r low-z, %r RAISIN1, %r RAISIN2'%(count_lowz, count_raisin1, count_raisin2))
    plt.text(0.05, y_interval[0]+0.2, r'($\Omega_m=$%r, $\omega=$%r, $\sigma_{v_{\rm pec}}=$%r km/s)'%(OmMFix, 
                         wFix, sigma_vpec))
    if PrintRMS: plt.text(0.05, y_interval[1]-0.35, 'RMS = %r'%round(rms_total,3))

# For low-z sample only
elif LowzSample and RAISIN_1_Sample ==False and RAISIN_2_Sample==False:
    plt.text(0.03, y_interval[1]-0.2, '%r low-z'%(count_lowz))
    plt.text(0.03, y_interval[0]+0.2, r'($\Omega_m=$%r, $\omega=$%r, $\sigma_{v_{\rm pec}}=$%r km/s)'%(OmMFix, 
                         wFix, sigma_vpec))
    if PrintRMS: plt.text(0.05, y_interval[1]-0.35, 'RMS = %r'%round(rms_total,3))
    

plt.grid()

plt.xlim(x_interval)
plt.ylim(y_interval)

plt.xlabel('Redshift')
plt.ylabel(r'$\mu - \mu_{\Lambda{\rm CDM}}$', fontsize=16)

# Title:
if BandType == 'Optical': plt.title('Hubble residual. Optical only')
elif BandType == 'OpticalNIR': plt.title('Hubble residual.  Optical+NIR')

if count_lowz >0: label_0 = 'lowz'
if count_raisin1 >0: label_1 = 'RAISIN1'
if count_raisin2 >0: label_2 = 'RAISIN2'
    
plt.savefig(DirSaveOutput+'HubbleDiag_%s_%s_%s_Om%r_w%r_%s_.png'%(label_0,
                                    label_1,label_2,OmMFix,wFix,BandType), dpi=180)
plt.close()

----

# Hubble diagram for SNANA-SALT2 fit

In [55]:
zcmbUpperLim = 0.6 

# For the Hubble diagram:
xlimPlots = 0.0015, zcmbUpperLim+0.003 
if zcmbUpperLim == 0.6: ylimPlots = 29.8, 38
else: ylimPlots = 29.8, 38.1 

# For the Hubble residual
ylimPlots_residual = -1, 1

In [56]:
OmMFix = 0.27;  wFix = -1

In [57]:
zcmb_np = Mu_raisin1Snana['f2']
mu_np   = Mu_raisin1Snana['f25']
res_mu_np = Mu_raisin1Snana['f25'] - DistanceMuVector(zcmb_np, OmMFix, wFix, HoFix)
err_mu_np = Mu_raisin1Snana['f26']

In [58]:
#    Theoretical values

# Array plot the -theoretical- (spectroscopic) distance modulus 
nbins1= 51
z1 = np.linspace(xlimPlots[0], xlimPlots[1], nbins1)
mu0 = DistanceMuVector(z1, OmMFix, wFix, HoFix)

In [59]:
#    Theoretical values

# Array plot the -theoretical- (spectroscopic) distance modulus 
nbins1= 51
z1 = np.linspace(xlimPlots[0], xlimPlots[1], nbins1)
mu0 = DistanceMuVector(z1, OmMFix, wFix, HoFix)

#----------------------------------------

plt.plot(z1, mu0)
plt.errorbar(zcmb_np, mu_np, yerr=err_mu_np, fmt='.')
plt.grid(True)
plt.savefig(DirSaveOutput+'Hubble_SALT2_.png')
plt.close()

# Hubble residual for SNANA-SALT2 fit

In [60]:
#    Theoretical values

# Array plot the -theoretical- (spectroscopic) distance modulus 
nbins1= 51
z1 = np.linspace(xlimPlots[0], xlimPlots[1], nbins1)
mu1 = np.zeros(len(z1))

#----------------------------------------

plt.plot(z1, mu1)
plt.errorbar(zcmb_np, res_mu_np, yerr=err_mu_np, fmt='.')
plt.grid(True)
plt.savefig(DirSaveOutput+'HubbleResidual_SALT2_.png')
plt.close()